# Road Traffic Accidents in Switzerland

Our project goal is to scrap all traffic accidents from the accidents map from http://map.donneesaccidents.ch/

## Data scraping strategy

Accessing http://map.donneesaccidents.ch/, wich redirects to : <br>
https://map.geo.admin.ch/?topic=vu&lang=fr&bgLayer=ch.swisstopo.pixelkarte-grau&layers=ch.astra.unfaelle-personenschaeden_alle&layers_timestamp=&catalogNodes=1318


Postman parses the following parameters : 
<code>
topic:vu
lang:en
bgLayer:ch.swisstopo.pixelkarte-grau
layers:ch.astra.unfaelle-personenschaeden_alle
layers_timestamp:
catalogNodes:1318
</code>

The most important one is layers:ch.astra.unfaelle-personenschaeden_alle.<br>
It is the layer that contains all the geo-information dots on "Accidents with personal injury" which is the selected data layer.
<img src="layer_selector.png">

Selection all kinds of accidents returns the following :<br>
<img src="layer_selector_all.png">
with layer parameters :<br>
layers:<br>
    &nbsp;ch.astra.unfaelle-personenschaeden_alle,<br>
    &nbsp;ch.astra.unfaelle-personenschaeden_getoetete,<br>
    &nbsp;ch.astra.unfaelle-personenschaeden_fussgaenger,<br>
    &nbsp;ch.astra.unfaelle-personenschaeden_fahrraeder,<br>
    &nbsp;ch.astra.unfaelle-personenschaeden_motorraeder<br>
layers_timestamp:,,,,<br>

Now we want every data for each layer. By selecting a dot on the map, it queries the related data to the server.
What we want to do is selecting all the entries in the map to retrieves all data. This is done by ctrl clicking the whole area.

This makes a query for each "layers" parameter :
<code>
geometry:443999.04209536605,39001.6733318335,870499.0420953662,303001.67333183356
geometryFormat:geojson
geometryType:esriGeometryEnvelope
imageDisplay:1536,759,96
lang:en
layers:all:<i>LAYER_PARAM</i>
mapExtent:269999.04209536605,9501.673331833561,1037999.042095366,389001.67333183356
returnGeometry:true
tolerance:5
</code><br>
But doesn't select all dots on map, so let's try the "load more results" button on a 'accidetns with fatalities' layer, we get :
<code>
geometry:443999.04209536605,39001.6733318335,870499.0420953662,303001.67333183356
geometryFormat:geojson
geometryType:esriGeometryEnvelope
imageDisplay:1536,759,96
lang:en
layers:all:ch.astra.unfaelle-personenschaeden_getoetete
mapExtent:136199.04209536605,-28148.32666816644,1134599.042095366,465201.67333183356
<b>offset:200</b>
returnGeometry:true
tolerance:5
</code>
Pressing load more until no more possible give offset=1200 (for a total of 1337 objects) i.e it loads data entries 200 by 200

## JSON Data scraping

In [159]:
import requests
import json

import numpy as np
import pandas as pd


import pprint
#from bs4 import BeautifulSoup

In [160]:
pp = pprint.PrettyPrinter(indent=4)

In [161]:
def import_data():
    url_0="https://api3.geo.admin.ch/rest/services/all/MapServer/identify?geometry=446000.0000000001,37750,860500.0000000002,317750.00000000006&geometryFormat=geojson&geometryType=esriGeometryEnvelope&imageDisplay=1536,759,96&lang=fr&layers=all:ch.astra.unfaelle-personenschaeden_getoetete&mapExtent=276000,250,1044000,379750&returnGeometry=true&tolerance=5"
    url_1="https://api3.geo.admin.ch/rest/services/all/MapServer/identify?geometry=446000.0000000001,37750,860500.0000000002,317750.00000000006&geometryFormat=geojson&geometryType=esriGeometryEnvelope&imageDisplay=1536,759,96&lang=fr&layers=all:ch.astra.unfaelle-personenschaeden_getoetete&mapExtent=276000,250,1044000,379750&offset=200&returnGeometry=true&tolerance=5"
    r0=requests.get(url_0)
    r1=requests.get(url_1)
    json_data0=(json.loads(str(r0.text))).get('results')
    json_data1=(json.loads(str(r1.text))).get('results')
    data = json_data0+json_data1 #merge two lists into one
    return data

In [162]:
#Remove all items we are not interested in
def clean_data(data_list):
    
    
    data_list_copy = data_list.copy()
    for item in data_list_copy : 
        properties = item.get('properties')
        del properties['accidentday_de']
        del properties['accidentday_it']
        del properties['accidenttype_de']
        del properties['accidenttype_it']
        del properties['severitycategory_it']
        del properties['severitycategory_de']
        
        
    return data_list_copy

In [174]:
def reformat_data(data_list):
    data_list_reformat = []
    
    for item in data_list:
        properties = item.get('properties')
        del item['properties']
        item = dict(item, **properties)
        data_list_reformat.append(item)
    
    return data_list_reformat
    
    

In [ ]:
#import and clean the data
json_data_clean = reformat_data(clean_data(import_data()))

In [157]:
print("each request gets 201 entry points")
print(len(json_data0))
print(len(json_data1))

print("Data entry example :\n")

pp.pprint(json_data_clean[0])

each request gets 201 entry points
201
201
402
Data entry example :

{   'accidentday_fr': 'lundi / 08h-09h / décembre 2015',
    'accidenttype_fr': 'dérapage ou perte de maîtrise',
    'accidenttypecode': 0,
    'accidentyear': 2015,
    'bbox': [692392.0, 186750.0, 692392.0, 186750.0],
    'canton': 'UR',
    'featureId': '2A28E2E05D0C01D6E0530A839427ED82',
    'fsocommunecode': '1206',
    'geometry': {'coordinates': [[692392.0, 186750.0]], 'type': 'MultiPoint'},
    'geometryType': 'Feature',
    'id': '2A28E2E05D0C01D6E0530A839427ED82',
    'label': 'Schleuder- oder Selbstunfall',
    'layerBodId': 'ch.astra.unfaelle-personenschaeden_getoetete',
    'layerName': 'Accidents mortels',
    'roadtype_de': 'Hauptstrasse',
    'roadtype_fr': 'route principale',
    'roadtype_it': 'Strada principale',
    'roadtypecode': 432,
    'severitycategory_fr': 'accident avec tués',
    'severitycategorycode': 'UGT',
    'type': 'Feature'}


In [154]:
df = pd.DataFrame.from_dict(json_data_clean)
df.set_index('id', inplace=True)
df

,accidentday_fr,accidenttype_fr,accidenttypecode,accidentyear,bbox,canton,featureId,fsocommunecode,geometry,geometryType,label,layerBodId,layerName,roadtype_de,roadtype_fr,roadtype_it,roadtypecode,severitycategory_fr,severitycategorycode,type
id,,,,,,,,,,,,,,,,,,,,
2A28E2E05D0C01D6E0530A839427ED82,lundi / 08h-09h / décembre 2015,dérapage ou perte de maîtrise,0,2015,"[692392.0, 186750.0, 692392.0, 186750.0]",UR,2A28E2E05D0C01D6E0530A839427ED82,1206,"{'coordinates': [[692392.0, 186750.0]], 'type'...",Feature,Schleuder- oder Selbstunfall,ch.astra.unfaelle-personenschaeden_getoetete,Accidents mortels,Hauptstrasse,route principale,Strada principale,432,accident avec tués,UGT,Feature
295D2DCE053C0016E0530A83942706A9,mardi / 19h-20h / décembre 2015,dérapage ou perte de maîtrise,0,2015,"[611712.0, 153780.0, 611712.0, 153780.0]",BE,295D2DCE053C0016E0530A83942706A9,0563,"{'coordinates': [[611712.0, 153780.0]], 'type'...",Feature,Schleuder- oder Selbstunfall,ch.astra.unfaelle-personenschaeden_getoetete,Accidents mortels,andere,autre,Altro,439,accident avec tués,UGT,Feature
2B085C4F46C30048E0530A8394274815,mardi / 14h-15h / décembre 2015,accident impliquant des piétons,8,2015,"[618619.0, 205951.0, 618619.0, 205951.0]",BE,2B085C4F46C30048E0530A8394274815,0955,"{'coordinates': [[618619.0, 205951.0]], 'type'...",Feature,Fussgängerunfall,ch.astra.unfaelle-personenschaeden_getoetete,Accidents mortels,Nebenstrasse,route secondaire,Strada secondaria,433,accident avec tués,UGT,Feature
299BEA3F80B002E2E0530A8394275D79,lundi / 09h-10h / décembre 2015,accident impliquant des piétons,8,2015,"[754250.0, 225823.0, 754250.0, 225823.0]",SG,299BEA3F80B002E2E0530A8394275D79,3271,"{'coordinates': [[754250.0, 225823.0]], 'type'...",Feature,Fussgängerunfall,ch.astra.unfaelle-personenschaeden_getoetete,Accidents mortels,Nebenstrasse,route secondaire,Strada secondaria,433,accident avec tués,UGT,Feature
29238FBC0DA20210E0530A8394278688,jeudi / 15h-16h / décembre 2015,dérapage ou perte de maîtrise,0,2015,"[587730.0, 226729.0, 587730.0, 226729.0]",BE,29238FBC0DA20210E0530A8394278688,0449,"{'coordinates': [[587730.0, 226729.0]], 'type'...",Feature,Schleuder- oder Selbstunfall,ch.astra.unfaelle-personenschaeden_getoetete,Accidents mortels,Nebenstrasse,route secondaire,Strada secondaria,433,accident avec tués,UGT,Feature
2A3DFE253486016AE0530A839427F4FD,mardi / 13h-14h / décembre 2015,dérapage ou perte de maîtrise,0,2015,"[491174.0, 113858.0, 491174.0, 113858.0]",GE,2A3DFE253486016AE0530A839427F4FD,6609,"{'coordinates': [[491174.0, 113858.0]], 'type'...",Feature,Schleuder- oder Selbstunfall,ch.astra.unfaelle-personenschaeden_getoetete,Accidents mortels,Hauptstrasse,route principale,Strada principale,432,accident avec tués,UGT,Feature
2AB2C7ECFCC90064E0530A83942737C3,lundi / 16h-17h / décembre 2015,accident impliquant des piétons,8,2015,"[534100.0, 154412.0, 534100.0, 154412.0]",VD,2AB2C7ECFCC90064E0530A83942737C3,5591,"{'coordinates': [[534100.0, 154412.0]], 'type'...",Feature,Fussgängerunfall,ch.astra.unfaelle-personenschaeden_getoetete,Accidents mortels,Hauptstrasse,route principale,Strada principale,432,accident avec tués,UGT,Feature
25C042211A7900B4E0530A839427843D,dimanche / 01h-02h / décembre 2015,dérapage ou perte de maîtrise,0,2015,"[596401.0, 240513.0, 596401.0, 240513.0]",JU,25C042211A7900B4E0530A839427843D,6708,"{'coordinates': [[596401.0, 240513.0]], 'type'...",Feature,Schleuder- oder Selbstunfall,ch.astra.unfaelle-personenschaeden_getoetete,Accidents mortels,Autobahn,autoroute,Autostrada,430,accident avec tués,UGT,Feature
2763C98279C80090E0530A839427A667,dimanche / 21h-22h / décembre 2015,accident impliquant des piétons,8,2015,"[691292.0, 248323.0, 691292.0, 248323.0]",ZH,2763C98279C80090E0530A839427A667,0197,"{'coordinates': [[691292.0, 248323.0]], 'type'...",Feature,Fussgängerunfall,ch.astra.unfaelle-personenschaeden_getoetete,Accidents mortels,Nebenstrasse,route secondaire,Strada secondaria,433,accident avec tués,UGT,Feature


1) Accidents par rapport au temps<br>
2) Corrélation nombre/type d'accident avec les endroits (Valais ivresse)<br>
3) Tracker des anomalies (fin/début d'une série d'accident) et essayer d'en trouver la cause<br>